In [ ]:
import numpy as np
import cv2 as cv
# camera paramters
f=3997.684
doffs=124.343
b=193.001
w=2964
h=2000
#loading right and left image using opencv function
imgL = cv.imread('C:/Users/user/OneDrive/Pictures/Camera Roll/im0 2.png',0)
imgR = cv.imread('C:/Users/user/OneDrive/Pictures/Camera Roll/im1 2.png',0)
imgC=  cv.imread('C:/Users/user/OneDrive/Pictures/Camera Roll/im0 2.png')
#obtaining disparity 
stereo = cv.StereoSGBM_create(minDisparity=0,numDisparities=256, blockSize=17,P1 =2500,P2 = 50,uniquenessRatio=4,speckleWindowSize=4,
speckleRange=4,disp12MaxDiff = 20)
disparity = stereo.compute(imgL,imgR)
#appling gaussain blur to make image smooth 
#disparity = cv.GaussianBlur(disparity,(10,10),0.1)
#assigning camera parameter to calibration martix usimg numpy library
Q=np.array([[f, 0, 0, 0], [0, f, 0,0],[0, 0, f,0],[0, 0,0,1]])
#obtaining 3D coordinates of points 
points=cv.reprojectImageTo3D(disparity,Q)
#obtaining colour of points 
colour=cv.cvtColor(imgC,cv.COLOR_BGR2RGB)
#thresholding points with very lareg depth or zero disparity
points=points[disparity>0]
colour= colour[disparity>0]
#creating point cloud using .ply file 
#defining elements and property of elements inside ply header 
ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d 
        property float x
        property float y
        property float z
        property uint8 red
        property uint8 green
        property uint8 blue
        end_header
        '''
# defining pointcloud function and writing element and its properties 
def pointcloud(vertices, color, filename):
    #using np.hstack function arranging properties in (x,y,z,red,green,blue) format of every vertic
    color= color.rshape(-1,3)
    vertices = np.hstack([vertices.reshape(-1,3),color])
    #opening and writing in .ply file 
    with open(filename, 'w') as f:
        f.write(ply_header %dict(vert_num=len(vertices)))
        np.savetxt(f,vertices,'%f %f %f %d %d %d')
#specifing file path and calling pointcloud function 
file = 'C:/Users/user/OneDrive/Desktop/New folder/stereocontruction1.ply'
pointcloud(points, colour,file)